In [1]:
!pip install transformers datasets peft accelerate scipy

# Process data

In [2]:
import json
import pandas as pd

In [3]:
departments = pd.read_csv('./data/department', sep='\t', encoding_errors='ignore')
print(departments.shape)
departments.head()

(26, 7)


,dept_code,division,name,office_number,building_code,chair_code,blurb
0,ENGL,Arts and Humanities,English,300,WNTP,lstaley,English study at Hudson unlocks rich opportuni...
1,SOAN,Social Sciences,Sociology and Anthropology,390,ALMN,eshever,Join in the ongoing comparative study of human...
2,MATH,Natural Sciences and Mathematics,Mathematics,290,FRLY,dschult,Explore the intrinsic elegance of humanitys tr...
3,ARTS,Arts and Humanities,Art and Art History,387,WHET,emarlowe,"Examine the history, theory, and practice of t..."
4,PHAS,Natural Sciences and Mathematics,Physics and Astronomy,331,FSC,egalvez,The physics and astronomy department explores ...


In [4]:
dept = dict(zip(departments['dept_code'], departments['name']))

In [5]:
with open('./data/students.json', 'r') as f:
    students = json.load(f)

with open('./data/responses.json', 'r') as f:
    responses = json.load(f)

len(students), len(responses)

(14, 14)

In [6]:
students_short = []
for i in range(len(students)):
    std_og = students[i]
    students_short.append({
        'student_name': std_og['student_name'],
        # 'student_gender': std_og['student_gender'],
        'student_year': std_og['student_year'],
        'student_major': dept[std_og['student_major']]
        # 'student_enrollment': [
        #     {'course_name': c['course_name'], 'grade': c['grade']}
        #     if c['currently_taking'] == False else
        #     {'course_name': c['course_name']}
        #     for c in std_og['student_enrollment']
        # ]
    })

In [7]:
train_data = []
for i in range(len(students)):
    train_data.append({
        'input': json.dumps(students_short[i]),
        'output': json.dumps(responses[i])
    })

In [8]:
with open('./data/train.jsonl', 'w') as f:
    for item in train_data:
        f.write(json.dumps(item) + '\n')

# Tune model

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
import torch

In [10]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [11]:
data = load_dataset("json", data_files={"train": "data/train.jsonl"})["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
def tokenize(example):
    prompt = f"### Instruction:\n{example['input']}\n\n### Response:\n{example['output']}"
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=1024)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

dataset = data.map(tokenize, remove_columns=["input", "output"])

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [14]:
example = data[0]
prompt = f"### Instruction:\n{example['input']}\n\n### Response:\n{example['output']}"
print(prompt)

### Instruction:
{"student_name": "Christina Edwards", "student_year": 1, "student_major": "Economics"}

### Response:
{"issue": "Academic difficulty and adjustment challenges in the first year of college", "meeting_request": {"purpose": "Struggling academically and feeling overwhelmed", "description": "Hi, my name is Christina and I'm a first-year Economics major. Last semester was really rough \u2014 I took a full load of courses, including several upper-level Econ classes, and ended up doing poorly across the board. My grades were all in the low 60s, and it's been discouraging. I\u2019m currently in a new semester with more demanding Econ courses and some electives, and I still feel like I\u2019m falling behind or missing something. I\u2019m not sure if I chose the right major, and I\u2019ve been feeling anxious about keeping up and whether I even belong here. I\u2019m reaching out because I\u2019d like some help figuring out how to get on track academically, feel more confident, an

In [15]:
base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

In [16]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)

In [17]:
training_args = TrainingArguments(
    output_dir="./tinyllama-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=20,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

In [18]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

<ipython-input-19-7333e3d8cd82>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,1.862300
20,1.732800
30,1.644600
40,1.444300
50,1.397200
60,1.224300
70,1.207300
80,1.153900
90,1.120000
100,1.054200


TrainOutput(global_step=280, training_loss=1.0078747476850236, metrics={'train_runtime': 109.8861, 'train_samples_per_second': 2.548, 'train_steps_per_second': 2.548, 'total_flos': 1781630112890880.0, 'train_loss': 1.0078747476850236, 'epoch': 20.0})

In [76]:
model.save_pretrained("./tinyllama-lora")
tokenizer.save_pretrained("./tinyllama-lora")

('./tinyllama-lora-2/tokenizer_config.json',
 './tinyllama-lora-2/special_tokens_map.json',
 './tinyllama-lora-2/tokenizer.model',
 './tinyllama-lora-2/added_tokens.json',
 './tinyllama-lora-2/tokenizer.json')

# Inference

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
import json

In [22]:
MODEL_PATH = "./tinyllama-lora"
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [23]:
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base, MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [24]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

In [25]:
with open('./data/students_test.json', 'r') as f:
    students_test = json.load(f)
len(students_test)

44

In [26]:
students_test_short = []
for i in range(len(students_test)):
    std_og = students_test[i]
    students_test_short.append({
        'student_name': std_og['student_name'],
        # 'student_gender': std_og['student_gender'],
        'student_year': std_og['student_year'],
        'student_major': dept[std_og['student_major']],
        # 'student_enrollment': [
        #     {'course_name': c['course_name'], 'grade': c['grade']}
        #     if c['currently_taking'] == False else
        #     {'course_name': c['course_name']}
        #     for c in std_og['student_enrollment']
        # ]
    })

In [27]:
student_test = json.dumps(students_test_short[0])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Marie Burton", "student_year": 3, "student_major": "Educational Studies"}\n\n### Response:\n'

In [28]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "Marie Burton", "student_year": 3, "student_major": "Educational Studies"}

### Response:
{"issue": "Struggling with course selection and academic anxiety in the final year of education studies", "meeting_request": {"purpose": "Help with college planning", "description": "Hi, I'm Marie, a third-year Education Studies student. I\u2019ve been doing well academically, but lately I\u2019ve been feeling overwhelmed by the number of courses I\u2019ve been considering and whether I should take on more advanced classes or focus on something new. I\u2019m also experiencing some academic anxiety, which is making it hard to make decisions. I\u2019d appreciate help understanding what schools I should be applying to, what courses are good fit, and how to balance academics with personal growth."}, "meeting_report": {"summary": "Maria is experiencing academic stress and is assisting a student with navigating the final year of education studies course selection.", "k

In [29]:
student_test = json.dumps(students_test_short[1])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Amanda Lynch", "student_year": 2, "student_major": "English"}\n\n### Response:\n'

In [30]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "Amanda Lynch", "student_year": 2, "student_major": "English"}

### Response:
{"issue": "Struggling with writing and academic performance in English and literature courses", "meeting_request": {"purpose": "Need help improving writing skills in my major", "description": "Hi, I'm Amanda, a second-year English major. Last semester was really tough academically, and I ended up with a low GPA. One of my courses was English literature, and I struggled to understand the texts and even understand the concepts. Now I'm in a major that requires writing, and I'm having a hard time writing even basic essays. I'm not sure how to improve my writing skills or how to make progress in my major. I'm really struggling and feel like I'm falling behind. I'm hoping to talk to someone about how to improve my writing in college and in my major."}, "meeting_report": {"summary": "Amanda is struggling with writing in her major and is experiencing academic setbacks.", "key_conce

In [32]:
student_test = json.dumps(students_test_short[3])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Kevin Hernandez", "student_year": 1, "student_major": "Chemistry"}\n\n### Response:\n'

In [33]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "Kevin Hernandez", "student_year": 1, "student_major": "Chemistry"}

### Response:
{"issue": "First-year student struggling with coursework and academic expectations", "meeting_request": {"purpose": "Need help understanding chemistry concepts", "description": "Hi, I\u2019m Kevin, a first-year Chemistry major. I\u2019m excited to be taking chemistry classes, but I\u2019m still struggling to understand the concepts behind molecules, atoms, and reactions. Last semester, I took a few chemistry courses and got a C- in each one. I\u2019m noticing that a lot of my classmates are doing better than me, and I\u2019m worried that I\u2019m falling behind. I\u2019m still trying to figure out what I\u2019m doing wrong and who I\u2019m messing up in this major. I\u2019d like to talk to someone about how to improve my academic performance and understanding of chemistry."}, "meeting_report": {"summary": "First-year student struggling with chemistry coursework, academi

In [35]:
student_test = json.dumps(students_test_short[4])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "James Ross", "student_year": 3, "student_major": "Theater"}\n\n### Response:\n'

In [36]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "James Ross", "student_year": 3, "student_major": "Theater"}

### Response:
{"issue": "Struggling with academic performance in the theater major", "meeting_request": {"purpose": "Need help improving my academic performance in the theater major", "description": "Hi, I\u2019m James, a third-year theater student. I\u2019ve taken a mix of theater and film courses, and while I\u2019ve enjoyed the theater classes, I\u2019ve found it harder than expected to keep up with the academic demands of the major. My grades have been falling, and I\u2019m worried I\u2019ll fall behind even more if I keep doing the theater courses. I\u2019m not sure if I should be in the theater major or not, and I\u2019m feeling overwhelmed and unsure of what to do."}, "meeting_report": {"summary": "James is experiencing academic pressure due to a mix of theater and film coursework in the theater major, but is unsure of his academic path and is feeling anxious.", "key_concerns": ["Dec

In [40]:
student_test = json.dumps(students_test_short[8])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Kristina Mendez", "student_year": 3, "student_major": "Environmental Studies"}\n\n### Response:\n'

In [41]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "Kristina Mendez", "student_year": 3, "student_major": "Environmental Studies"}

### Response:
{"issue": "Struggling with course selection and academic direction in environmental studies", "meeting_request": {"purpose": "Need help finding a suitable major in environmental studies", "description": "Hi, I'm Kristina, a third-year Environmental Studies major. I've been really enjoying the coursework I'm taking, but I'm still not sure what kind of major I should choose. I'm interested in environmental policy, but I also want to explore ecological issues and human-nature relations. I'm not sure if environmental studies covers all of these areas, or if I should consider other majors like Biology or Public Policy. I'm feeling overwhelmed and not sure how to find the right fit. I'm hoping to talk to someone who can help me explore my options and help me decide which major will best suit my interests and skills."}, "meeting_report": {"summary": "Kristina is un

In [45]:
student_test = json.dumps(students_test_short[12])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Trevor Gordon", "student_year": 3, "student_major": "Economics"}\n\n### Response:\n'

In [46]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "Trevor Gordon", "student_year": 3, "student_major": "Economics"}

### Response:
{"issue": "Struggling with coursework and preparing for graduate school in economics", "meeting_request": {"purpose": "Help with college course selection and understanding economics", "description": "Hi, I'm Trevor, a third-year Economics major. Despite enjoying the course material, I'm finding it challenging to balance economics with other coursework and personal responsibilities. I\u2019m interested in graduating strong but not sure how to prepare for graduate school in economics or how to make progress in my major. I\u2019m also worried about feeling overwhelmed by the material and whether I\u2019m doing well enough to apply to schools. I\u2019d appreciate guidance on how to improve my course selection, understanding economics, and my academic performance."}}


In [47]:
pipe2 = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=3072)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

In [49]:
student_test = json.dumps(students_test_short[13])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Mary Harrison", "student_year": 1, "student_major": "Biology"}\n\n### Response:\n'

In [50]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "Mary Harrison", "student_year": 1, "student_major": "Biology"}

### Response:
{"issue": "First-year anxiety and academic stress", "meeting_request": {"purpose": "Feeling overwhelmed by first-year Biology classes and academic expectations", "description": "Hi, I\u2019m Mary, a first-year Biology major. I\u2019ve taken two difficult biology courses this semester and already feel like I\u2019m falling behind. I\u2019m not particularly good at math, and the material in these classes is much harder than what I\u2019ve been used to. I\u2019m also in a new academic environment, with a lot of pressure to perform well and get good grades. I\u2019m worried that I\u2019m not measuring up and will fail these classes, which will make me feel even more isolated and anxious. I\u2019d like some support dealing with academic stress and first-year anxiety in biology. I\u2019d also want to talk about how to build a support network in the biology department and in the u

In [52]:
student_test = json.dumps(students_test_short[14])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Leah Roberts", "student_year": 4, "student_major": "Writing and Rhetoric"}\n\n### Response:\n'

In [53]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "Leah Roberts", "student_year": 4, "student_major": "Writing and Rhetoric"}

### Response:
{"issue": "Uncertainty about career path or writing-related major impact in future job opportunities", "meeting_request": {"purpose": "Help choosing between writing and rhetoric majors", "description": "Hi, I'm Leah, a senior majoring in Writing and Rhetoric. I've been really enjoying my \u201cmajor\u201d and have been exploring different writing-related options, but I\u2019m not sure which major will be the best fit for my long-term career goals. I\u2019m interested in writing-related jobs like writing for nonprofits, editing for publications, or teaching writing, but I\u2019m not sure if a rhetoric or writing-focused major will prepare me for those types of work. I\u2019m also interested in exploring other fields like public health or education that could benefit from writing skills, but I\u2019m not sure which fields might need writing in some capacity. I\u20

In [54]:
student_test = json.dumps(students_test_short[15])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Diane Meyer", "student_year": 2, "student_major": "Political Science"}\n\n### Response:\n'

In [55]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "Diane Meyer", "student_year": 2, "student_major": "Political Science"}

### Response:
{"issue": "Uncertainty about career path and potential impact of political studies in college", "meeting_request": {"purpose": "Help deciding between major and career path after majoring in politics", "description": "Hi, I'm Diane, a second-year Political Science major. Despite enjoying the coursework and discussions, I'm not sure what I want to do after college. I'm interested in politics but not sure if I want to work in government or advocacy. I'm also worried about finding a job in a competitive field like politics or media after majoring in a non-traditional field. I'm currently exploring career paths that involve policy or advocacy, but I'm not sure what's feasible or what I should be looking for. I'm looking for guidance on what courses and majors might be a good fit for me, and how to decide between Political Science and other fields like Business, Human Res

In [57]:
student_test = json.dumps(students_test_short[17])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Christopher Robles", "student_year": 1, "student_major": "Political Science"}\n\n### Response:\n'

In [58]:
out = pipe(prompt)[0]["generated_text"]
print(out)

### Instructions:
{"student_name": "Christopher Robles", "student_year": 1, "student_major": "Political Science"}

### Response:
{"issue": "Academic difficulty in first-year Political Science students", "description": "Hi, I\u2019m Christopher, a first-year Political Science major. So far, I\u2019ve been taking mostly introductory courses and feeling like I\u2019m not really understanding what I\u2019m doing. Some of my professors have been helpful, but I\u2019m also feeling overwhelmed by the material and worried I\u2019m going to fail. I\u2019m not sure if I\u2019m even on track to graduate, and I\u2019m worried I\u2019ll be behind academically."}, {"resolution": "Based on Christopher\u2019s initial experiences, I\u2019d recommend a one-on-one session to discuss his academic progress and current course load. We can discuss strategies to stay on track, including breaking down the material into smaller units and using tools like note-taking apps or academic calendars. We can also revie

In [59]:
student_test = json.dumps(students_test_short[18])
prompt = f"### Instructions:\n{student_test}\n\n### Response:\n"
prompt

'### Instructions:\n{"student_name": "Daniel Cook", "student_year": 4, "student_major": "English"}\n\n### Response:\n'

In [60]:
out = pipe(prompt)[0]["generated_text"]
print(out)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


### Instructions:
{"student_name": "Daniel Cook", "student_year": 4, "student_major": "English"}

### Response:
{"issue": "Struggling with writing and academic performance in English and literature courses", "meeting_request": {"purpose": "Need help improving writing skills in my major", "description": "Hi, I'm Daniel, a senior majoring in English. Last semester was really tough academically, and I struggled with writing assignments and even some of my literature courses. I\u2019m noticing that I\u2019m falling behind in my major now, and I\u2019m worried I\u2019ll fail my major if I don\u2019t improve. I\u2019m open to talking about how to improve both academically and personally through writing, and I\u2019d like some guidance on how to get started. I\u2019d like to work with a tutor or mentor to work on a writing schedule and review materials, and I\u2019m open to different writing genres or styles depending on what works best."}, "meeting_report": {"summary": "Daniel is experiencin

In [61]:
from datasets import Dataset

In [62]:
ds = Dataset.from_list(students_test_short)

In [65]:
def format_prompt(example):
    example_dict = {k: example[k] for k in example.keys()}
    prompt = f"""### Instruction:
{json.dumps(example_dict)}

### Response:"""
    return {"prompt": prompt}

In [66]:
ds = ds.map(format_prompt)

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [74]:
outputs = pipe(ds["prompt"], batch_size=1)

In [75]:
for output in outputs:
    print(output[0]["generated_text"] + '\n')

### Instruction:
{"student_name": "Marie Burton", "student_year": 3, "student_major": "Educational Studies"}

### Response:
{"issue": "Struggling with academic performance in a liberal arts degree program", "meeting_request": {"purpose": "Feeling overwhelmed by coursework and uncertain if I can keep up", "description": "Hi, I\u2019m Marie, a third-year Education major. I\u2019ve taken a mix of liberal arts and education-related courses, and I\u2019m really struggling to keep up with the content and pace. Some of the classes are really heavy on writing or research methods, and other ones are more practical like teaching methods or curriculum design. I feel like I\u2019m trying to learn everything at once, but it\u2019s overwhelming. I\u2019m not sure if I\u2019m even doing well enough to graduate on time, and I\u2019m starting to panic. I\u2019d really appreciate help understanding what I\u2019m doing well or poorly in, how to pace myself better, and if there\u2019s anything I can do to